In [1]:
import os

# # Set the proxy environment variables
# os.environ['http_proxy'] = 'http://127.0.0.1:7890'
# os.environ['https_proxy'] = 'http://127.0.0.1:7890'

In [2]:
import ee
import datetime
import json
import os
import itertools
import sys
import re
from glob import glob
from tqdm.auto import tqdm
from dfply import *

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE
import ast

In [3]:
ee.Initialize()

In [4]:
# append upper folder into sys-path during run time so we can
# import our own moduls
sys.path.append('../../')

In [5]:
from BackGround_modules.Class_2_Classify_Fourier_Img   import Classification
from BackGround_modules.Class_3_Calculate_the_accuracy import Accuracy_assesment

##### Prepare basic parameters
| Image    | Sample          | Classification |
|:---------|:---------------|:--------------|
| dongbei  | dongbei         | dongbei        |
| huabei   | huabei          | huabei         |
| huadong  | ★zhongnan       | huadong        |
| xibei    | ★xibei&★zhongnan| xibei          |
| xinan    | ★huadong       | xinan          |
| zhongnan | zhongnan        | zhongnan       |


In [6]:
# define the region to analysis
region_cns = ['华东',   '东北',   '中南',    '华北',  '西北', '西南']
region_ens = ['huadong','dongbei','zhongnan','huabei','xibei','xinan']

In [7]:
# define the region in this single run
region_cn = '西南'
region_en = 'xinan'

# do you want to use the sample pts from other region?
# other_sample = 'huadong'
other_sample = region_en

In [8]:
# define the years to be classified
year_name  = [f'{i}_{i+2}' for i in range(1990,2023,3)]

# define the number of total periods
num_invarient = len(year_name)

In [9]:
year_name

['1990_1992',
 '1993_1995',
 '1996_1998',
 '1999_2001',
 '2002_2004',
 '2005_2007',
 '2008_2010',
 '2011_2013',
 '2014_2016',
 '2017_2019',
 '2020_2022']

In [10]:
# import China shp boundary
Region =  ee.FeatureCollection("users/wangjinzhulala/China_built_up/01_Boundary_shp/China_zone")\
            .filterMetadata('NAME1','equals',region_cn)

# Define the path to Sample_pt
sample_img_path = f'users/wangjinzhulala/China_built_up/04_sample_train_test_split'
export_path = 'users/wangjinzhulala/China_built_up/05_primary_classification'

##### Fetch all imgs

In [11]:
# read all asset_df into a list
asset_list = [pd.read_csv(f) for f in glob('../../File_trainsition/gee_asset_china/*.csv')]

# concate all asset_df into one df
asset_df = pd.concat(asset_list)
asset_df.sort_values(['region','type','year'],inplace=True)

In [12]:
asset_df

,year,type,path,region
0,1990_1992,Fourier,projects/earthengine-legacy/assets/users/bthom...,dongbei
1,1993_1995,Fourier,projects/earthengine-legacy/assets/users/bthom...,dongbei
2,1996_1998,Fourier,projects/earthengine-legacy/assets/users/bthom...,dongbei
3,1999_2001,Fourier,projects/earthengine-legacy/assets/users/bthom...,dongbei
4,2002_2004,Fourier,projects/earthengine-legacy/assets/users/bthom...,dongbei
...,...,...,...,...
11,2008_2010,NDVI_NDBI_EVI,projects/earthengine-legacy/assets/users/fhane...,zhongnan
12,2011_2013,NDVI_NDBI_EVI,projects/earthengine-legacy/assets/users/fhane...,zhongnan
13,2014_2016,NDVI_NDBI_EVI,projects/earthengine-legacy/assets/users/fhane...,zhongnan
14,2017_2019,NDVI_NDBI_EVI,projects/earthengine-legacy/assets/users/fhane...,zhongnan


##### Prepare input_image for classification

Some additional treatment for the last two periods of Landsat and Sentinel imgs

In [13]:
# import Landsat img
Landsat_df = asset_df[(asset_df['type']=='Landsat_cloud_free')&(asset_df['region']==region_en)]
Landsat_img = [ee.Image(i) for i in  Landsat_df['path'].values]

# import Fourier img   
Fourier_df  = asset_df[(asset_df['type']=='Fourier')&(asset_df['region']==region_en)]
Fourier_img = [ee.Image(i) for i in  Fourier_df['path'].values]

# import Normalized img   
Normalized_df  = asset_df[(asset_df['type']=='NDVI_NDBI_EVI')&(asset_df['region']==region_en)]
Normalized_img = [ee.Image(i) for i in  Normalized_df['path'].values]

# prepare the climate data
# because the climate data ends in 2019, we will use the 2019 data if requring future data than 2019
if num_invarient > 10:
    climate_year = year_name[:10] + [year_name[9]]*(num_invarient-10)
Climate_mean = [ee.Image(f"users/China_bulit/01_Data_preparation/Climate_Mean_{year}")
                for year in climate_year]

# Import DEM/SLOPE Img
DEM   = [ee.Image("USGS/SRTMGL1_003").rename('DEM')] * num_invarient
SLOPE = [ee.Terrain.slope(DEM).rename('SLOPE')] * num_invarient

In [14]:
# stack all Imput_Img together
Stack_img = [ee.Image(img) for img in zip(Landsat_img,
                                           Fourier_img,
                                           Normalized_img,
                                           Climate_mean,
                                           DEM,
                                           SLOPE)]

##### Prepare input sample points

In [15]:
# Get the training sample
Training_sample = [ee.FeatureCollection(f"{sample_img_path}/Training_sample_{other_sample}_{year}")
                   for year in year_name]

##### Stack input_img and input_sample together

In [16]:
# first put all inputs together
stack_year   = year_name
stack_img    = Stack_img 
stack_sample = Training_sample 

# stack all ingredients together
Stack_img_sample = list(zip(stack_year,stack_img,stack_sample))

#### Create the classification instances

In [17]:
# import the inbands-combination 
in_band_df_before_2020 = pd.read_csv('../Sub_Process_7_Before_classification_Sensitivity_test/Result/In_bands_combination.csv')
in_band_2017_2019  = in_band_df_before_2020 >> mask(X.year_range=='2017_2019')

if num_invarient > 10:
    in_band_df = pd.concat([in_band_df_before_2020]+[in_band_2017_2019]*(num_invarient-10),0)
    in_band_df['year_range'] = year_name

# selections for in-band combination
in_band_selection =[#'Spectrum',
                    #'Spectrum_Normalize',
                    #'Spectrum_Normalize_Fourier',
                    #'Spectrum_Normalize_Fourier_Terrain',
                    'Spectrum_Normalize_Fourier_Terrain_Meterology']

C:\Users\Administrator\AppData\Local\Temp\ipykernel_17308\965096381.py:6: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  in_band_df = pd.concat([in_band_df_before_2020]+[in_band_2017_2019]*(num_invarient-10),0)


In [18]:
# A three-layer iteration to loopthrough [inbands/year/10-randome-split] individualy

Classificatioin_result = []

# The first-layer loop to go througth each inbands
for input_bands in in_band_selection:

    # The second-layer loop to go througth each year-periods
    for i,input_variable in enumerate(Stack_img_sample):
        
        
        # Fetch basic parameters
        classificaiton_tpye = input_bands
        year                = input_variable[0]
        input_img           = input_variable[1]
        sample_pt           = input_variable[2]

        # here determine which bands are involved in the classification
        # because the list is a string represented format, here use ast 
        # module to conver it back to a Python list
        in_features         = ast.literal_eval(in_band_df.iloc[i][input_bands])
        

        # The third-layer to go through each-randomnes
        for seed in range(10):

            #_______________________________Deploy the random-forest classification______________________
            # Instatiate the class with a name.
            classification = Classification(year_name      = year,
                                            Verified_point = sample_pt,
                                            Input_img      = input_img,
                                            Input_band     = in_features,
                                            Tree_num       = 100,
                                            seed           = seed,
                                            classProperty  = 'Built')


            # Get the classified img
            classified_img = classification.classification_img


            # Get the classified samples
            train_sample_classified = classification.Train_sample_classification
            test_sample_classified  = classification.Test_sample_classification

            #__________________________________Add all classification result into a list____________________________
            Classificatioin_result.append((classificaiton_tpye,year,seed,classified_img,test_sample_classified))

In [19]:
# convert classificaiton instances into a dataframe
Classification_instances_df =  pd.DataFrame(Classificatioin_result,
                                            columns=['classificaiton_tpye','year','seed','classified_img',
                                                     'test_sample_classified'])
Classification_instances_df

,classificaiton_tpye,year,seed,classified_img,test_sample_classified
0,Spectrum_Normalize_Fourier_Terrain_Meterology,1990_1992,0,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.FeatureCollection({\n ""functionInvocationV..."
1,Spectrum_Normalize_Fourier_Terrain_Meterology,1990_1992,1,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.FeatureCollection({\n ""functionInvocationV..."
2,Spectrum_Normalize_Fourier_Terrain_Meterology,1990_1992,2,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.FeatureCollection({\n ""functionInvocationV..."
3,Spectrum_Normalize_Fourier_Terrain_Meterology,1990_1992,3,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.FeatureCollection({\n ""functionInvocationV..."
4,Spectrum_Normalize_Fourier_Terrain_Meterology,1990_1992,4,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.FeatureCollection({\n ""functionInvocationV..."
...,...,...,...,...,...
105,Spectrum_Normalize_Fourier_Terrain_Meterology,2020_2022,5,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.FeatureCollection({\n ""functionInvocationV..."
106,Spectrum_Normalize_Fourier_Terrain_Meterology,2020_2022,6,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.FeatureCollection({\n ""functionInvocationV..."
107,Spectrum_Normalize_Fourier_Terrain_Meterology,2020_2022,7,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.FeatureCollection({\n ""functionInvocationV..."
108,Spectrum_Normalize_Fourier_Terrain_Meterology,2020_2022,8,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.FeatureCollection({\n ""functionInvocationV..."


#### Add one of the classified img to map

In [20]:
# define parameters of classification for show in map
bands_in_type = in_band_selection[0]
year_in       = '2020_2022'


# get the classification img
one = (Classification_instances_df >> mask(X.classificaiton_tpye == bands_in_type,
                                  X.year == year_in,
                                  X.seed == 5))['classified_img'].values[0]

In [21]:
# add the selected classification in map
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map.centerObject(Region,6)
Map.addLayer(one.clip(Region),{'min':0,'max':1},'One')
Map

Map(center=[30.069204468133314, 95.82418599870017], controls=(WidgetControl(options=['position', 'transparent_…

##### Export the classified_img

In [180]:
# Region = ee.FeatureCollection("users/wangjinzhulala/China_built_up/01_Boundary_shp/Classificatioin_boundary_patch")

In [181]:
selecte_year = '2020_2022' # or None

if selecte_year:
    Classification_instances_df = Classification_instances_df >> mask(X.year==selecte_year)

    
for idx,item in Classification_instances_df.iterrows():
     
    Classification_type = item[0]
    year = item[1]
    seed = item[2]
    img  = item[3].toInt8()
    
    
    export_name = f'{Classification_type}_{region_en}_{year}_{seed}'
    
    #export to GEE asset    
    task = ee.batch.Export.image.toAsset(   image          = img,
                                            description    = export_name,
                                            assetId        = f'{export_path}/{export_name}',
                                            region         = Region.geometry().bounds(),
                                            scale          = 30,
                                            maxPixels      = int(1e13)) 
    
    task.start()
    
    
    print(f'Exporting {export_name} successful!')

Exporting Spectrum_Normalize_Fourier_Terrain_Meterology_xinan_2020_2022_0 successful!
Exporting Spectrum_Normalize_Fourier_Terrain_Meterology_xinan_2020_2022_1 successful!
Exporting Spectrum_Normalize_Fourier_Terrain_Meterology_xinan_2020_2022_2 successful!
Exporting Spectrum_Normalize_Fourier_Terrain_Meterology_xinan_2020_2022_3 successful!
Exporting Spectrum_Normalize_Fourier_Terrain_Meterology_xinan_2020_2022_4 successful!
Exporting Spectrum_Normalize_Fourier_Terrain_Meterology_xinan_2020_2022_5 successful!
Exporting Spectrum_Normalize_Fourier_Terrain_Meterology_xinan_2020_2022_6 successful!
Exporting Spectrum_Normalize_Fourier_Terrain_Meterology_xinan_2020_2022_7 successful!
Exporting Spectrum_Normalize_Fourier_Terrain_Meterology_xinan_2020_2022_8 successful!
Exporting Spectrum_Normalize_Fourier_Terrain_Meterology_xinan_2020_2022_9 successful!
